# Handle exception in Pyspark 🛠️

**Lời nói đầu:**

🔹 Khi làm việc với pyspark cho tác vụ xử lý dữ liệu lớn, mình đã gặp không ít lỗi về phần cài đặt cũng như cấu hình framework, nhất là khi làm trên local machine, (remote thì khỏi nói, nó support tận răng, chẳng hạn như colab 🙄). Đây là sẽ tutorial đúc kết ra nhiều kiến thức xử lý lỗi sau nhiều ngày vật lộn với cái pyspark này.

🔹 Lỗi (error) và ngoại lệ (exception) trong PySpark rất dài. Làm việc với PySpark giống như kiểu bạn phải làm việc với combo Python error + exception Java vậy.

![](https://th.bing.com/th/id/OIP.vTD1E70SWGFDQEGOJHHAyAHaC9?rs=1&pid=ImgDetMain)

## Small Cases 📇📇📇

Dưới đây là một vài lỗi nho nhỏ mà mình gặp phải ở lần đầu tiên làm việc với pyspark (đúng hơn là tự mình tạo ra 😢) 

**🚫 Cài đặt thiếu biến môi trường trên local**

Yeah lỗi này cũng là do lần đầu tiên "ngựa ngựa" cài đặt trên local machine, trong khi người người nhà nhà thì toàn làm trên google colab 😅.

Lỗi này rất dài và sẽ thông báo như sau:

```Error
Py4JJavaError: An error occurred while calling z:org.apache.spark.api.python.PythonRDD.collectAndServe.
: org.apache.spark.SparkException: Job aborted due to stage failure: Task 7 in stage 1.0 failed 1 times, most recent failure: Lost task 7.0 in stage 1.0 (TID 19) (host.docker.internal executor driver): java.io.IOException: Cannot run program "": CreateProcess error=87, The parameter is incorrect
```

Lỗi này xuất hiện khi mình thao tác với action count() trên rdd của pyspark. Nó xuất phát từ `Py4J`, một thư viện giúp Python giao tiếp với JVM (Java Virtual Machine). Khi chạy một action như count(), PySpark sẽ submit lệnh từ Python sang Java (JVM). Nếu có lỗi xảy ra trong Java, nó sẽ được báo lại dưới dạng `Py4JJavaError` trong Python. 

Ở case của mình, nguyên nhân chủ yếu là mình chạy trên jupyter notebook, mặc dù đã cài biến môi trường `JAVA_HOME`, `SPARK_HOME`, và `PYSPARK_PYTHON` đầy đủ trên máy tính nhưng jupyter notebook vẫn không nhận diện được đường dẫn khiến cho một vài task bị fail.

Cách giải quyết:

In [1]:
import os
os.environ["JAVA_HOME"] = "C:\\Program Files\\Java\\jdk-11"
os.environ["SPARK_HOME"] = "C:\\spark\\spark-3.5.4-bin-hadoop3"
os.environ["PYSPARK_PYTHON"] ="C:\\Users\\nam\\anaconda3\\envs\\py3_9\\python.exe"

❗❗❗ Lưu ý, tùy thuộc vào phiên bản python mà bạn đang cài đặt nhưng nhìn chung thì bạn sẽ phải mò mẫm xem cái  file python.exe nằm ở đâu. Còn với java, nếu bạn cài đặt phiên bản thấp hơn chẳng hạn java 8 thì chỉ cần đổi lại thành jdk-8 là được

**🚫 Vô ý stop một context và thực thi với rdd**

Lỗi này khá cơ bản, mình nghĩ rằng mình và cũng như nhiều beginer khi làm việc với các hệ thống xử lý dữ liệu lớn như spark sẽ gặp phải. Nhiều trường hợp mà một cách thần kỳ nào đó ta lại vô thức dừng session để rồi gặp phải thông báo sau:

```Error
AttributeError: 'NoneType' object has no attribute 'sc'
```
Ví dụ:


**✅ Code không lỗi:**

In [2]:
from pyspark.sql import SparkSession

# Khởi tạo một session
spark = SparkSession.builder \
    .appName("Datacamp Pyspark Tutorial") \
    .config("spark.memory.offHeap.enabled","true") \
    .config("spark.memory.offHeap.size","5g").getOrCreate()

# Lấy Context từ session
sc = spark.sparkContext

# tạo một rdd
rdd_words = sc.parallelize(
    [
    'Scalar',
    'java',
    'hadoop',
    'spark',
    'akka',
    'spark and hadoop',
    'pyspark',
    'pyspark and spark'
    ]
)

# show kết quả
rdd_words.collect()
rdd_words.count()

8

**❌ Code bị lỗi:**

In [3]:
from pyspark import SparkContext

sc.stop()

# tạo một rdd
rdd_words = sc.parallelize(
    [
    'Scalar',
    'java',
    'hadoop',
    'spark',
    'akka',
    'spark and hadoop',
    'pyspark',
    'pyspark and spark'
    ]
)

# show kết quả
rdd_words.count()

# AttributeError: 'NoneType' object has no attribute 'sc'

AttributeError: 'NoneType' object has no attribute 'sc'